# Check synchronization of local and cloud sqlite databases

In [ ]:
from lndb import login, init, settings, delete
import os

In [ ]:
login("testuser1@lamin.ai")

In [ ]:
init(storage="s3://lndb-setup-ci")

Get the paths to the cloud and local sqlite databases.

In [ ]:
sqlite_file = settings.instance._sqlite_file

In [ ]:
sqlite_file

In [ ]:
cache_file = settings.instance.storage.cloud_to_local_no_update(sqlite_file)

In [ ]:
cache_file

Delete the local sqlite database file.

In [ ]:
cache_file.unlink()

In [ ]:
assert not cache_file.exists()

Access to the session restores the local version of sqlite database by downloading it from the cloud.

In [ ]:
settings.instance.session()

In [ ]:
assert cache_file.exists()

If the local sqlite database is older than the cloud one, the cloud database replaces the local sqlite database file.

In [ ]:
cloud_mtime = sqlite_file.stat().st_mtime

In [ ]:
cloud_mtime

In [ ]:
os.utime(cache_file, times=(cloud_mtime - 1, cloud_mtime - 1))

In [ ]:
assert cache_file.stat().st_mtime < sqlite_file.stat().st_mtime

In [ ]:
settings.instance.session()

In [ ]:
assert cache_file.stat().st_mtime == sqlite_file.stat().st_mtime

In [ ]:
# clean up
settings.instance._sqlite_file.unlink()
delete("lndb-setup-ci")